## Document selection for manual anntoation

Manual annotation is a process of marking documents with annotations that can be used to train or test a natural language processing system. We are using brat annotation tool to acquire annotations for your final project. 
Setting up a project within brat tool requires a set of files:
- *.txt files - Text files to be annotated
- *.ann files - Annotation files that will store annotations
- annotation.conf: annotation type configuration
- visual.conf: annotation display configuration
- tools.conf: annotation tool configuration
- kb_shortcuts.conf: keyboard shortcut tool configuration

Each annotation project typically defines its own annotation.conf. Defining visual.conf, tools.conf and kb_shortcuts.conf is not necessary, and the system falls back on simple default visuals, tools and shortcuts if these files are not present. 

First we need to select files to be annotated. For our project we will draw documents from MIMIC database.

Let's connect to the database (remember, the password to the database is the same as the user name):


In [ ]:
import pymysql
import pandas as pd
import getpass
conn = pymysql.connect(host="mysql",
                       port=3306,user="jovyan",
                       passwd=getpass.getpass("Enter MySQL passwd for jovyan"),db='mimic2')

We have connected to the database. Let's see the tables that we have to work with.

In [ ]:
pd.read_sql("SELECT table_name, table_rows FROM information_schema.tables where table_schema='mimic2'", conn)

There are quite a few tables to explore and you should do it for your project. The documents are in "noteevents" table. Let's see what columns this table has. 

In [ ]:
pd.read_sql("SELECT column_name, is_nullable, column_type FROM information_schema.columns WHERE table_name = 'noteevents'",conn)


Note that there are only two fields that are not nullable, which means that cannot be blank, so they have a potential to be the primary key. Let's take a look at what the data looks like:

In [ ]:
pd.read_sql("SELECT subject_id, charttime, text  from noteevents where text is not null limit 10",conn)


 The table does not have document id, but has "subject_id" as a patient identifier. This is a limitation of a Demo dataset. NOTEID column exists in the full MIMIC database, but not in the current demo version. We will have to come up with a unique name for the documents.
 
### Select documents from the databse

For this demo, we will select 10 random documents from "noteevents" table that contain a keyword that we are interested in.

In [ ]:
docs_text = pd.read_sql("SELECT subject_id, text from noteevents   where text like '% fever %' order by rand() limit  10",conn)
docs_text

Iterate through the documents to see what they look like.

In [ ]:
for index, row in docs_text.iterrows():
    print(index, row[0], row[1])

### Wrting documents into files

To keep your documents separate from everyone elses, enter your UNID. 

In [1]:
unid = 'jianlins'

Check the folder just to see that you have prepared your workspace.

In [2]:
%%bash  -s "$unid"
echo ~/BRAT/$1/*
ls   ~/BRAT/$1/ 

/home/jianlins/BRAT/jianlins/*


ls: cannot access '/home/jianlins/BRAT/jianlins/': No such file or directory


Create a folder for your project. Let's name it "Project_1"

In [4]:
%%bash  -s "$unid"
mkdir   ~/BRAT/$1/
mkdir   ~/BRAT/$1/Project_1 
echo ~/BRAT/$1/*
ls   ~/BRAT/$1/ 

/home/jianlins/BRAT/jianlins/Project_1
Project_1


The project is created, so now we can write our files. Text files from "text" field will contain the note text, and the file name will be a combination of subject_id and the row index from the data frame. This combination will create a unique name for your files. ".ann" files are blank at first.

In [ ]:
path = "/home/"+str(unid)+"/BRAT/"+str(unid)+"/Project_1"
for index, row in docs_text.iterrows():
    new_file_path_txt = path+"/"+str(row[0]) + "_" + str(index) + ".txt" 
    new_file_path_ann = path+"/"+str(row[0]) + "_" + str(index) + ".ann" 
    print(new_file_path_txt)
    print(new_file_path_ann)   
    

In [ ]:
for index, row in docs_text.iterrows():
    new_file_path_txt = path+"/"+str(row[0]) + "_" + str(index) + ".txt" 
    new_file_path_ann = path+"/"+str(row[0]) + "_" + str(index) + ".ann" 
    f=open(new_file_path_txt, "w")
    f.write(row[1])
    f.close()
    f=open(new_file_path_ann, "w")
    f.write("")
    f.close()

Check to make sure that the files got written to the correct folder.

In [ ]:
%%bash  -s "$unid"  
ls   ~/BRAT/$1/Project_1/*

Now you have files to annotate.